<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/07_Bankkunden_gehen_weg/Bankkunden_gehen_weg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Abwanderung von Bankkunden**

Die Bank begann jeden Monat Kunden zu verlassen. Bankvermarkter haben herausgefunden: Es ist billiger, aktuelle Kunden zu behalten, als neue anzuziehen.
Es ist notwendig vorherzusagen, ob der Kunde die Bank in naher Zukunft verlässt oder nicht. Uns werden historische Daten über das Verhalten der Kunden und die Kündigung von Verträgen mit der Bank vorgelegt.
Es ist notwendig, ein Modell mit einem extrem hohen F1-Maßwert von mindestens 0,59 zu erstellen.

# Datenexploration

In [1]:
# Wir importieren die notwendigen Bibliotheken
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import f1_score 
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

In [2]:
# Wir laden Tabellen mit Daten
data = pd.read_csv('/content/Churn.csv')

In [3]:
# Wir werden die erhaltenen Daten studieren
data.info()
display(data.shape)
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


(10000, 14)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


**Folgerungen**

Die Tabelle enthält 10.000 Zeilen und 14 Spalten. Die Spalte 'Tenure' enthält Nullwerte. Die Spalten 'RowNumber', 'CustomerID', 'Surname' sind für die Analyse nicht erforderlich.

# Datenvorverarbeitung

In [4]:
# Wir berechnen den arithmetischen Mittelwert und den Median in der Spalte 'Tenure'.
print('mean', data['Tenure'].mean())
print('median', data['Tenure'].median())

mean 4.997690023099769
median 5.0


In [5]:
# Das arithmetische Mittel und der Median für die Spalte 'Tenure' sind praktisch gleich.
# Wir werden die Nullwerte mit dem Median füllen.
median_days_employed = data['Tenure'].median()
data['Tenure'] = data['Tenure'].fillna(median_days_employed)

# Wir werden prüfen, ob die Tabelle Nullwerte enthält.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [6]:
# Wir werden nicht benötigte Spalten für die Analyse entfernen
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace = True)

display(data.shape)
data.head()

(10000, 11)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


**Folgerungen**

Nullwerte sind ausgefüllt, unnötige Spalten werden entfernt.

# Datenaufbereitung

In [7]:
# Wir werden die direkte Codierung auf die Tabelle anwenden
data_ohe = pd.get_dummies(data, drop_first=True)
display(data_ohe.shape)
display(data_ohe.head())

(10000, 12)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


In [8]:
# Wir werden die Zielzeichen und Bedingungen hervorheben
target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis=1)

# Wir werden die Daten in Stichproben zerlegen.
features_train_1, features_valid, target_train_1, target_valid = train_test_split(features, target, test_size=0.20, random_state=12345)
features_train, features_test, target_train, target_test = train_test_split(features_train_1, target_train_1, test_size=0.25, random_state=12345)

# Wir prüfen, ob die Daten korrekt verteilt wurden.
print('{:.0%}'.format(target_train.shape[0]/target.shape[0]))
print('{:.0%}'.format(target_valid.shape[0]/target.shape[0]))
print('{:.0%}'.format(target_test.shape[0]/target.shape[0]))

60%
20%
20%


# Aufgabenforschung

## Modelltraining

In [9]:
# Wir werden das Modell trainieren, den F1-Indikator messen
model = LogisticRegression(solver='liblinear', random_state = 12345) 
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.09896907216494845

In [10]:
# Wir werden das Modell trainieren, den F1-Indikator messen
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.5165876777251184

In [12]:
# Wir werden das Modell trainieren, den F1-Indikator messen
model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.5697674418604651

## Bekämpfung des Ungleichgewichts

In [13]:
# Wir werden die Gewichte der Klassen ausgeglichen machen und sehen, wie sich F1 ändert
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.4451313755795981

In [14]:
# Wir werden die Gewichte der Klassen ausgeglichen machen und sehen, wie sich F1 ändert
model = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.5151148730350664

In [15]:
# Wir werden die Gewichte der Klassen ausgeglichen machen und sehen, wie sich F1 ändert
model = RandomForestClassifier(random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_score(target_valid, predicted_valid)

0.5481927710843373

## Erhöhen und Verringern der Stichprobe

In [16]:
# Wir werden die Probe erhöhen
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 20)

In [17]:
# Wir werden sehen, wie sich der F1-Wert nach der Erhöhung der Stichprobe geändert hat
model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

0.3518747424804285

In [19]:
# Wir werden sehen, wie sich der F1-Wert nach der Erhöhung der Stichprobe geändert hat
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

0.484394506866417

In [20]:
# Wir werden sehen, wie sich der F1-Wert nach der Erhöhung der Stichprobe geändert hat
model = RandomForestClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

0.5942408376963352

In [21]:
# Wir werden die Probe verringern
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.2)

In [22]:
# Wir werden sehen, wie sich der Wert von F1 nach der Reduzierung der Stichprobe geändert hat
model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

0.44271844660194176

In [23]:
# Wir werden sehen, wie sich der Wert von F1 nach der Reduzierung der Stichprobe geändert hat
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

0.4972200158856236

In [24]:
# Wir werden sehen, wie sich der Wert von F1 nach der Reduzierung der Stichprobe geändert hat
model = RandomForestClassifier(random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
display(f1_score(target_valid, predicted_valid))

0.5821517931609674

## Schwellenwert ändern

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

display(roc_auc_score(target_valid, probabilities_one_valid))

for threshold in np.arange(0, 0.5, 0.05):
    predicted_valid = probabilities_one_valid > threshold
    print(threshold, f1_score(target_valid, predicted_valid))